In [11]:
# import numpy as np
# from functions import scales_mean_matrix_factorization, ALSMatrixFactorization

# # make user-item matrix (calculating different "rating" metrics)

# # do matrix factorization (using different matrix factorization methods: Alternating Least Squares, Scale's Mean, others??)
# ratings = np.array([[0.5, 0, 4],
#                     [1, 3, 5]])


# res = scales_mean_matrix_factorization(ratings)
# print("res", res)


# # ALSmodel = ALSMatrixFactorization(5, 3, 0.5, ratings)
# # res = ALSmodel.fit()
# # print(res)

# # ALSmodel = ALSMatrixFactorization(15, 3, 0.5, ratings)
# # res = ALSmodel.fit()
# # print(res)

# ALSmodel = ALSMatrixFactorization(5, 3, 0.5, ratings)
# res = ALSmodel.fit()
# mse = ALSmodel.get_training_mse()
# print(mse)
# mape = ALSmodel.get_training_mape()
# print(mape)

import numpy as np
from functions import scales_mean_matrix_factorization, ALSMatrixFactorization
from sklearn.model_selection import train_test_split
import pandas as pd


# Load the user-item matrix
user_item_matrix = np.load('user_item_matrix.npy')

# Convert matrix to long-format DataFrame
rows, cols = np.where(user_item_matrix > 0)
data = pd.DataFrame({
    'user_id': rows,
    'item_id': cols,
    'count': user_item_matrix[rows, cols]
})

print("Long-format DataFrame:\n", data.head())

def stratified_split(data, test_size=0.2, random_state=42):
    train_list = []
    test_list = []

    for user_id, user_data in data.groupby('user_id'):
        if len(user_data) > 1:
            user_train, user_test = train_test_split(
                user_data, test_size=test_size, random_state=random_state)
            train_list.append(user_train)
            test_list.append(user_test)
        else:
            train_list.append(user_data)

    train_data = pd.concat(train_list)
    test_data = pd.concat(test_list)

    return train_data, test_data

# Perform the stratified split
train_df, test_df = stratified_split(data, test_size=0.2)

print("Training Set:\n", train_df.head())
print("Testing Set:\n", test_df.head())

# Convert back to user-item matrix form
num_users, num_items = user_item_matrix.shape
train_matrix = np.zeros((num_users, num_items))
test_matrix = np.zeros((num_users, num_items))

train_matrix[train_df['user_id'], train_df['item_id']] = train_df['count']
test_matrix[test_df['user_id'], test_df['item_id']] = test_df['count']

print("Training Matrix:\n", train_matrix)
print("Testing Matrix:\n", test_matrix)

def calculate_mse(actual, predicted):
    mask = actual != 0
    return np.mean((actual[mask] - predicted[mask]) ** 2)

def calculate_mape(actual, predicted):
    mask = actual != 0
    return np.mean(np.abs((actual[mask] - predicted[mask]) / actual[mask])) * 100

# ratings = np.array([[1, 0, 5],
#                     [1, 5, 5],
                    
#                     [5, 0, 1],
#                     [5, 1, 1],])

# Apply the model
ALSmodel = ALSMatrixFactorization(5, 3, 0.5, train_matrix)
ALSmodel.fit()

# Predict ratings for training and testing sets
train_predictions = ALSmodel.predict()
test_predictions = ALSmodel.predict()

# Evaluate the model
train_mse = calculate_mse(train_matrix, train_predictions)
train_mape = calculate_mape(train_matrix, train_predictions)

test_mse = calculate_mse(test_matrix, test_predictions)
test_mape = calculate_mape(test_matrix, test_predictions)

print("Training MSE:", train_mse)
print("Training MAPE:", train_mape)

print("Testing MSE:", test_mse)
print("Testing MAPE:", test_mape)

# print("User-item matrix loaded from 'user_item_matrix.npy'")
# print("User-item matrix shape:", user_item_matrix.shape)

# # Use the loaded matrix in your functions
# ratings = user_item_matrix  # or any subset or transformation you need

# res = scales_mean_matrix_factorization(ratings)
# print("res", res)

# ALSmodel = ALSMatrixFactorization(5, 3, 0.5, ratings)
# res = ALSmodel.fit()
# print(res)

# mse = ALSmodel.get_training_mse()
# print("Training MSE:", mse)

# mape = ALSmodel.get_training_mape()
# print("Training MAPE:", mape)





Long-format DataFrame:
    user_id  item_id  count
0        0      177    1.0
1        0      195    3.0
2        0      209    3.0
3        0      517    2.0
4        0      549    1.0
Training Set:
    user_id  item_id  count
0        0      177    1.0
7        0      774    2.0
2        0      209    3.0
4        0      549    1.0
3        0      517    2.0
Testing Set:
     user_id  item_id  count
1         0      195    3.0
5         0      765    2.0
9         1      793    1.0
15        3      581    2.0
17        4      403    2.0
Training Matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 2.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Testing Matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processo

AttributeError: 'ALSMatrixFactorization' object has no attribute 'calculate_mse'